# 0. Предварительная подготовка

In [2]:
import os
import pandas as pd
import requests
import textwrap
from transformers import pipeline

In [3]:
DATA_URL = "https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz"
DATA_PATH = "lenta-ru-news.csv.gz"

if not os.path.exists(DATA_PATH):
    print(f"Файл {DATA_PATH} не найден, скачиваю из {DATA_URL}")
    response = requests.get(DATA_URL, stream=True, timeout=60)
    if response.status_code != 200:
        raise RuntimeError(f"Не удалось скачать датасет: HTTP {response.status_code}")
    with open(DATA_PATH, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    print("Загрузка завершена")
else:
    print(f"Используем локальный файл {DATA_PATH}")

df = pd.read_csv(DATA_PATH)

# Удаляем строки с отсутствующим текстом
df.dropna(subset=["text"], inplace=True)

# Оставим только нужные колонки
df = df[["title", "text", "topic"]].copy()
print(df.info())
print()
print(df.head())

Используем локальный файл lenta-ru-news.csv.gz
<class 'pandas.core.frame.DataFrame'>
Index: 739346 entries, 0 to 739350
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   739346 non-null  object
 1   text    739346 non-null  object
 2   topic   739143 non-null  object
dtypes: object(3)
memory usage: 22.6+ MB
None

                                               title  \
0  Названы регионы России с самой высокой смертно...   
1  Австрия не представила доказательств вины росс...   
2       Обнаружено самое счастливое место на планете   
3  В США раскрыли сумму расходов на расследование...   
4  Хакеры рассказали о планах Великобритании зами...   

                                                text        topic  
0  Вице-премьер по социальным вопросам Татьяна Го...       Россия  
1  Австрийские правоохранительные органы не предс...        Спорт  
2  Сотрудники социальной сети Instagram проанализ...  Путешествия  
3  С на

# 1. Hugging Face и pipeline API

In [7]:
# Для проверки выберем 5 новостей
indices_to_summarize = [1, 40, 100, 500, 1000]
texts_from_dataset = df.loc[indices_to_summarize]["text"].tolist()

# Добавим 3 примера out-of-domain

# Стихи
poem = """Ночь, улица, фонарь, аптека,
          Бессмысленный и тусклый свет.
          Живи еще хоть четверть века -
          Всё будет так. Исхода нет.
          
          Умрешь - начнешь опять сначала
          И повторится всё, как встарь:
          Ночь, ледяная рябь канала,
          Аптека, улица, фонарь."""

# Переписка
texting = """А: Привет! Ты вчера до скольки сидел?
             Б: Привет. Почти до двух ночи.
             А: Жесть.
             Б: Не то слово. Зато, кажется, я нашел тот баг с авторизацией.
             А: Серьезно? Который два дня искали?
             Б: Да.
             А: И что там было?
             Б: Банально. В одном из env-файлов не хватало ключа.
             А: В смысле?
             Б: Ну, в .env.local он был, а в .env.prod – нет.
             А: И деплой шел с prod-конфигом?
             Б: Именно. А локально у всех все работало.
             А: Классика. Я же говорил, что надо конфиги проверить.
             Б: Говорил. Но кто ж думал, что так тупо."""

# Текст объявления с Авито (авторская орфография и пунктуация)
short_text = "Разминочный костюм BJORN DAEHLIE ,состояние новой вещи,без каких -либо следов износа.По бирке xl,но в реале L,замеры на фото,ориентируйтесь по ним.Произведено в России,но качества материала,пошива и лекала на высоте...Красивый яркий комбинезон."

texts_from_dataset.extend([poem, texting, short_text])

In [9]:
summarizer = pipeline("summarization", model="IlyaGusev/rut5_base_sum_gazeta")


# Функция для форматирования вывода 4 строки максимум по 120 символов
def wrapper(text: str) -> str:
    return textwrap.fill(text, width=120, max_lines=4, placeholder="...")


for i, text in enumerate(texts_from_dataset, start=1):
    # Генерируем саммари
    summary = summarizer(text, max_length=300, min_length=30, do_sample=False)

    # Вывод в читаемом формате
    print("\n" + "=" * 120 + "\n")
    print(f"--- Пример № {i} ---")
    print(f"Оригинал:\n{wrapper(text)}")
    print("\n")
    print(f"Суммаризация:\n{wrapper(summary[0]['summary_text'])}")
    print(f"\nИзменение размера: {len(text)} -> {len(summary[0]['summary_text'])}")

Device set to use cuda:0




--- Пример № 1 ---
Оригинал:
Австрийские правоохранительные органы не представили доказательств нарушения российскими биатлонистами антидопинговых
правил. Об этом сообщил посол России в Вене Дмитрий Любинский по итогам встречи уполномоченного адвоката дипмиссии с
представителями прокуратуры страны, передает ТАСС. «Действует презумпция невиновности. Каких-либо ограничений свободы
передвижения для команды нет», — добавили в посольстве. Международный союз биатлонистов (IBU) также не будет...


Суммаризация:
Австрийские правоохранительные органы не представили доказательств нарушения российскими биатлонистами антидопинговых
правил. Международный союз биатлонистов (IBU) также не будет применять санкции к российским биатлонистам. Международный
союз биатлонистов (IBU) также не будет применять санкции к российским биатлонистам.

Изменение размера: 1072 -> 321


--- Пример № 2 ---
Оригинал:
В Московском районе Санкт-Петербурга на продажу выставлена доля в 58-комнатной квартире. Об этом в свое

Your max_length is set to 300, but your input_length is only 89. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)




--- Пример № 5 ---
Оригинал:
Российский скелетонист Никита Трегубов победил на этапе Кубка мира в Латвии. Он обошел хозяина трассы Мартинса Дукурса,
сообщает ТАСС. По итогам двух заездов 23-летний россиянин показал результат 1 минута 41,87 секунды. Дукурс занял второе
место и отстал на 0,37 секунды. Третьим стал действующий обладатель Кубка мира южнокореец Юн Сон Бин (плюс 0,53). Для
Трегубова эта победа стала первой в карьере на этапах Кубка мира. На Олимпиаде 2018 года он завоевал серебряную...


Суммаризация:
Российский скелетонист Никита Трегубов победил на этапе Кубка мира в Латвии. 23-летний россиянин показал результат 1
минута 41,87 секунды и отстал на 0,37 секунды. Третьим стал действующий обладатель Кубка мира южнокореец Юн Сон Бин
(плюс 0,53). Третьим стал действующий обладатель Кубка мира южнокореец Юн Сон Бин (плюс 0,53).

Изменение размера: 1064 -> 327


Your max_length is set to 300, but your input_length is only 205. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=102)




--- Пример № 6 ---
Оригинал:
Ночь, улица, фонарь, аптека,           Бессмысленный и тусклый свет.           Живи еще хоть четверть века -
Всё будет так. Исхода нет.            Умрешь - начнешь опять сначала           И повторится всё, как встарь:
Ночь, ледяная рябь канала,           Аптека, улица, фонарь.


Суммаризация:
Ночь, улица, фонарь, аптека, Бессмысленный и тусклый свет. Живи еще четверть века - Всё будет так. Исхода нет. Исхода
нет. Исхода нет. Исхода нет. Исхода нет. Исхода нет. Исхода нет. Исхода нет. Исхода нет. Исхода нет. Исхода нет. Исхода
нет. Исхода нет. Исхода нет. Исхода нет. Исхода нет. Исхода нет. Исхода нет. Исхода нет.

Изменение размера: 297 -> 326


Your max_length is set to 300, but your input_length is only 92. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)




--- Пример № 7 ---
Оригинал:
А: Привет! Ты вчера до скольки сидел?              Б: Привет. Почти до двух ночи.              А: Жесть.              Б:
Не то слово. Зато, кажется, я нашел тот баг с авторизацией.              А: Серьезно? Который два дня искали?
Б: Да.              А: И что там было?              Б: Банально. В одном из env-файлов не хватало ключа.              А:
В смысле?              Б: Ну, в .env.local он был, а в .env.prod – нет.              А: И деплой шел с prod-конфигом?...


Суммаризация:
Вчера я нашел тот баг с авторизацией, который нашел я.env.local, а в одном из файлов не хватало ключа, а в одном из
файлов не хватало ключа. А в одном из файлов не хватало ключа, а в одном из env-файлов не хватало ключа.

Изменение размера: 661 -> 220


--- Пример № 8 ---
Оригинал:
Разминочный костюм BJORN DAEHLIE ,состояние новой вещи,без каких -либо следов износа.По бирке xl,но в реале L,замеры на
фото,ориентируйтесь по ним.Произведено в России,но качества материала,пошива 

# Выводы

Тестирование модели `IlyaGusev/rut5_base_sum_gazeta` показало:

1.  **Новости (целевой домен):**
    * Модель хорошо справляется с новостными статьями, корректно передавая их суть и ключевые факты.
    * Основной недостаток — периодические повторы предложений в сгенерированных саммари.

2.  **Другие домены (out-of-domain):**
    * Модель не справляется с нецелевыми форматами, что ожидаемо, так как она обучалась на новостях.
    * **Стихи:** Генерируется набор повторяющихся фраз.
    * **Диалог:** Попытка связного текста приводит к бессмыслице и искажению фактов.
    * **Объявление:** Короткий текст обрабатывается неверно, с генерацией нелогичных фрагментов.

# 2. Эмбеддинги трансформеров